In [41]:
! pip install --q athina

In [42]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ['ATHINA_API_KEY'] = userdata.get('ATHINA_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [43]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [44]:
# load data
from langchain.document_loaders import CSVLoader
loader = CSVLoader("./context.csv")
documents = loader.load()

In [45]:
# split documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

In [46]:
# initialize pinecone client
from pinecone import Pinecone as PineconeClient, ServerlessSpec
pc = PineconeClient(
    api_key=os.environ.get("PINECONE_API_KEY"),
)

In [48]:
# create index
pc.create_index(
        name='my-index',
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [49]:
# load index
index_name = "my-index"

In [52]:
import time
from langchain.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings
# create vectorstore
max_retries = 3  # Maximum number of retries
retry_delay = 1  # Initial delay in seconds

for attempt in range(max_retries):
    try:
        # create vectorstore
        vectorstore = Pinecone.from_documents(
            documents=documents,
            embedding=embeddings,
            index_name=index_name
        )
        break  # Exit loop if successful
    except RateLimitError as e:
        if attempt < max_retries - 1:  # Don't wait on the last attempt
            print(f"Rate limit hit, retrying in {retry_delay} seconds... (Attempt {attempt + 1}/{max_retries})")
            time.sleep(retry_delay)
            retry_delay *= 2  # Exponential backoff
        else:
            raise e  # Re-raise the exception if max retries reached

NameError: name 'RateLimitError' is not defined